In [58]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install langchain_pinecone


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain_pinecone from https://files.pythonhosted.org/packages/7c/1f/df38fcf4604d826dcc949fb8f36074a326ea70870dddcf9123cefa6a9275/langchain_pinecone-0.1.0-py3-none-any.whl.metadata


In [59]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
import os


In [4]:
!mkdir pdfs

In [76]:
loader=PyPDFDirectoryLoader("pdfs")

In [77]:
loader.load()

Ignoring wrong pointing object 1047 0 (offset 0)


[Document(page_content=' \nTIBCO BusinessWorks Performance Best Practices This document describes architectural concepts related to performance tuning for TIBCO BusinessWorks 5.x. Included are different tuning parameters, steps required to configure parameters, and best design techniques for better performance. The reader should focus on a real customer use case to understand different issues and solutions associated with each. The last section in this paper describes a typical performance management process including critical performance criteria, debugging and monitoring a typical project.  \nhttp://www.tibco.com Global Headquarters 3303 Hillview Avenue Palo Alto, CA 94304 Tel: +1 650-846-1000 Toll Free: 1 800-420-8450 Fax: +1 650-846-1005 ©Copyright 2005, TIBCO Software Inc. All rights reserved BCO, the TIBCO logo, The Power of Now, CO BusinessWorks, and TIBCO Software are trademarks or registered trademarks of TIBCO Software Inc. in the United States and/or other countries. All oth

In [78]:
data=loader.load()

Ignoring wrong pointing object 1047 0 (offset 0)


In [79]:
data[0]

Document(page_content=' \nTIBCO BusinessWorks Performance Best Practices This document describes architectural concepts related to performance tuning for TIBCO BusinessWorks 5.x. Included are different tuning parameters, steps required to configure parameters, and best design techniques for better performance. The reader should focus on a real customer use case to understand different issues and solutions associated with each. The last section in this paper describes a typical performance management process including critical performance criteria, debugging and monitoring a typical project.  \nhttp://www.tibco.com Global Headquarters 3303 Hillview Avenue Palo Alto, CA 94304 Tel: +1 650-846-1000 Toll Free: 1 800-420-8450 Fax: +1 650-846-1005 ©Copyright 2005, TIBCO Software Inc. All rights reserved BCO, the TIBCO logo, The Power of Now, CO BusinessWorks, and TIBCO Software are trademarks or registered trademarks of TIBCO Software Inc. in the United States and/or other countries. All othe

In [80]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [81]:
text_chunks=text_splitter.split_documents(data)

In [82]:
text_chunks

[Document(page_content='TIBCO BusinessWorks Performance Best Practices This document describes architectural concepts related to performance tuning for TIBCO BusinessWorks 5.x. Included are different tuning parameters, steps required to configure parameters, and best design techniques for better performance. The reader should focus on a real customer use case to understand different issues and solutions associated with each. The last section in this paper describes a typical performance management process including', metadata={'source': 'pdfs\\TIBCO_BW_BestPractices.pdf', 'page': 0}),
 Document(page_content='process including critical performance criteria, debugging and monitoring a typical project.', metadata={'source': 'pdfs\\TIBCO_BW_BestPractices.pdf', 'page': 0}),
 Document(page_content='http://www.tibco.com Global Headquarters 3303 Hillview Avenue Palo Alto, CA 94304 Tel: +1 650-846-1000 Toll Free: 1 800-420-8450 Fax: +1 650-846-1005 ©Copyright 2005, TIBCO Software Inc. All right

In [83]:
print(text_chunks[0].page_content)

TIBCO BusinessWorks Performance Best Practices This document describes architectural concepts related to performance tuning for TIBCO BusinessWorks 5.x. Included are different tuning parameters, steps required to configure parameters, and best design techniques for better performance. The reader should focus on a real customer use case to understand different issues and solutions associated with each. The last section in this paper describes a typical performance management process including


In [84]:
print(text_chunks[1].page_content)

process including critical performance criteria, debugging and monitoring a typical project.


In [85]:
print(text_chunks[2].page_content)

http://www.tibco.com Global Headquarters 3303 Hillview Avenue Palo Alto, CA 94304 Tel: +1 650-846-1000 Toll Free: 1 800-420-8450 Fax: +1 650-846-1005 ©Copyright 2005, TIBCO Software Inc. All rights reserved BCO, the TIBCO logo, The Power of Now, CO BusinessWorks, and TIBCO Software are trademarks or registered trademarks of TIBCO Software Inc. in the United States and/or other countries. All other product and company names and marks mentioned in this document are the property of their


In [86]:
len(text_chunks)

315

In [22]:
A=[19,20,21,22,23]

In [23]:
type(A)

list

In [24]:
len(A)

5

In [25]:
type(text_chunks)

list

In [87]:
import os
os.environ["OPENAI_API_KEY"]="sk-proj-iblH772yekDTmP3OFnizT3BlbkFJkonvRoQsC03Gvoz3eWWD"
os.environ["PINECONE_API_KEY"]="81754309-acf1-4ac8-bba0-28f4da9d5426"

In [88]:
embedding=OpenAIEmbeddings()

In [89]:
len(embedding.embed_query("How are you?"))

1536

In [90]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','81754309-acf1-4ac8-bba0-28f4da9d5426')



In [91]:
import pinecone

In [92]:
PINECONE_API_KEY

'81754309-acf1-4ac8-bba0-28f4da9d5426'

In [93]:
index_name='test'

In [94]:
index=pinecone.Index(api_key='81754309-acf1-4ac8-bba0-28f4da9d5426',host='https://test-x603g1g.svc.aped-4627-b74a.pinecone.io')

In [95]:
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embedding=embedding,index_name=index_name)

In [96]:
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embedding=embedding,index_name=index_name)

In [97]:
docsearch

In [67]:
query="DO PLANTS BREATHE?"

In [68]:
docs=docsearch.similarity_search(query)

In [69]:
docs

[Document(page_content='the cell.\n12.1 DO PLANTS BREATHE ?\nWell, the answer to this question is not quite so dir ect.  Y es, plants r equir e\nO2 for respiration to occur and they also give out CO2.  Hence, plants have\nsystems in place that ensure the availability of O2.  Plants, unlike animals,\nhave no specialised organs for gaseous exchange but they have stomata\nand lenticels for this purpose. There are several reasons why plants can\nget along without respiratory organs. First, each plant part takes care of'),
 Document(page_content='RESPIRATION  IN PLANTS 153\nAll of us breathe to live, but why is breathing so essential to life? What\nhappens when we breathe? Also, do all living organisms, including plants\nand microbes, breathe? If so, how?\nAll living organisms need energy for carrying out daily life activities,\nbe it absorption, transport, movement, reproduction or even breathing.\nWher e does all this ener gy come fr om? W e know we eat food for ener gy –'),
 Document(pag

In [70]:
llm=OpenAI()

In [71]:
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=docsearch.as_retriever())

In [98]:
query="What are best practice in TIBCO BW?"

In [99]:
qa.run(query)

" I don't know. The provided context doesn't give any information about the best practices for TIBCO BusinessWorks. "

In [ ]:
import sys
while True:
    user_input=input(f"Input Prompt: ")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa({'query': user_input})
    print(f"Answer: {result['result']}")

Input Prompt:  How to calculate MaxJobs and FlowLimit ?


Answer:  According to the provided context, calculating MaxJobs and FlowLimit for a process engine depends on factors such as the amount of memory reserved for the engine JVM, the maximum size of a process instance object, and the number of process instances available to run. It is also important to design a test suite before configuring anything.


Input Prompt:  what is EMS?


Answer:  EMS stands for TIBCO Enterprise Message Service. It is a messaging server that stores messages and ensures their delivery to the correct destination, allowing for efficient communication between different components of a system. 


Input Prompt:  What is MemorySavingMode?


Answer:  MemorySavingMode is a parameter in TIBCO BusinessWorks that can be set to True in order to improve memory footprint and performance for processes that manipulate large amounts of data. This setting allows the engine to release references to unused process data, allowing it to be garbage collected by the JVM.
